# Camino AI: Location Intelligence for AI Agents 🗺️🤖

Welcome to this comprehensive tutorial on **Camino AI**, a powerful location intelligence API that provides real-world grounding for AI agents. Unlike traditional web search APIs, Camino specializes in spatial reasoning, location-based queries, and journey planning—essential capabilities for agents that need to understand and navigate the physical world.

## What Makes Camino AI Special?

While tools like Tavily excel at web search and content extraction, Camino AI fills a critical gap in the AI agent ecosystem:

- **🌍 Real-World Grounding**: Connect AI agents to actual physical locations using OpenStreetMap data
- **🧠 Natural Language Understanding**: Convert queries like "quiet coffee shops for working" into structured location data
- **📍 Spatial Intelligence**: Calculate routes, distances, travel times, and spatial relationships
- **🎯 Context-Aware Search**: Understand intent and context (e.g., "romantic dinner spot" vs "business lunch")
- **🚶 Journey Planning**: Plan multi-stop trips with time and transport constraints
- **💰 Cost-Effective**: only $0.001 per call with a generous free tier

## Tutorial Overview

In this tutorial, you'll learn how to:
1. Set up and authenticate with Camino AI
2. Perform natural language location searches with AI-powered ranking
3. Calculate spatial relationships and routes between locations
4. Understand location context and surroundings
5. Plan complex journeys with multiple waypoints
6. Integrate Camino with AI agents via MCP (Model Context Protocol)

---

## Getting Started

### Prerequisites

1. **Sign up** for Camino AI at [https://app.getcamino.ai](https://app.getcamino.ai?utm_source=agents-towards-production&utm_medium=github&utm_campaign=tutorial) to get your API key
    - ![Select Google or Github to link your account](./assets/signup.png)
2. **Generage your first API key** from your Camino dashboard
    - ![Click Generate API Key](./assets/generateapikey.png)
    - ![Copy the generated key](./assets/saveapikey.png)
3. **Install dependencies** using the cells below

### Setting Up Your Environment

In [ ]:
# Install required dependencies
!pip install --upgrade camino-ai-sdk python-dotenv --quiet

In [ ]:
# Save your API key to .env file (replace with your actual key)
!echo "CAMINO_API_KEY=<your-camino-api-key>" >> .env

### Initialize the Camino Client

In [ ]:
import asyncio
import os
import json
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv
import getpass

from camino_ai import (
    CaminoAI,
    QueryRequest,
    RelationshipRequest,
    ContextRequest,
    JourneyRequest,
    RouteRequest,
    Coordinate,
    Waypoint,
    TransportMode,
    APIError,
)

# Load environment variables
load_dotenv()

# Get API key from environment or prompt user
if not os.environ.get("CAMINO_API_KEY"):
    os.environ["CAMINO_API_KEY"] = getpass.getpass("Enter your Camino API Key: ")

# Initialize the client
api_key = os.environ["CAMINO_API_KEY"]
client = CaminoAI(api_key=api_key)
print("✅ Camino AI client initialized successfully!")

## Part 1: Natural Language Location Search 🔍

Camino AI's query endpoint understands natural language and context. Let's explore how it interprets different types of location queries.

In [31]:
# Example 1: Simple location search near Brooklyn
# Using synchronous API for simplicity in notebook
response = client.query("coffee shops in Brooklyn")

print("🔍 Query: Coffee shops in Brooklyn")
print(f"⭕ Search results\n")

# Display results
if response.results:
    print(f"Found {response.total if hasattr(response, 'total') else len(response.results)} locations:\n")
    for i, place in enumerate(response.results[:5], 1):
        print(f"{i}. {place.name}")
        if hasattr(place, 'address') and place.address:
            print(f"   📍 Address: {place.address}")
        if hasattr(place, 'confidence'):
            print(f"   📊 Confidence Score: {place.confidence:.2f}")
        print()

🔍 Query: Coffee shops with wifi in Brooklyn
⭕ Search results

Found 772 locations:

1. La Fondita
   📊 Confidence Score: 1.00

2. Mocha Hookah
   📊 Confidence Score: 0.90

3. Blank Street Coffee
   📊 Confidence Score: 0.80

4. Southside Coffee
   📍 Address: 652 6th Avenue
   📊 Confidence Score: 0.70

5. Doughnut Plant
   📍 Address: 198 Bedford Avenue Brooklyn
   📊 Confidence Score: 0.60



### Context-Aware Searches

Camino understands context and intent, not just keywords. Let's see how it handles different contexts for the same type of location.

In [57]:
# Example 2: Advanced Query with Parameters using QueryRequest
central_park = Coordinate(lat=40.7831, lon=-73.9712)

# Context-aware restaurant searches
contexts = [
    "romantic italian dinner for anniversary",
    "sandwich business lunch meeting",
    "late night food after concert"
]

print("🎯 Context-Aware Restaurant Searches near Central Park\n")
print("="*60)

for context in contexts:
    query_request = QueryRequest(
        query=f"restaurants for {context}",
        lat=central_park.lat,
        lon=central_park.lon,
        radius=2500,
        limit=3
    )
    
    try:
        response = client.query(query_request)
        
        print(f"\n🔍 Query: '{context}'")
        if response.results:
            print("📍 Top recommendations:")
            for place in response.results[:2]:
                print(f"   • {place.name}")
                if hasattr(place, 'confidence'):
                    print(f"     Confidence: {place.confidence:.2f}")
    except APIError as e:
        print(f"❌ Query failed: {e.message}")
    print("-"*60)

🎯 Context-Aware Restaurant Searches near Central Park


🔍 Query: 'romantic italian dinner for anniversary'
📍 Top recommendations:
   • Giovanni
     Confidence: 1.00
   • Bosino
     Confidence: 0.90
------------------------------------------------------------

🔍 Query: 'sandwich business lunch meeting'
📍 Top recommendations:
   • Trudy's Ice Cream
     Confidence: 1.00
   • Giovanni
     Confidence: 0.90
------------------------------------------------------------

🔍 Query: 'late night food after concert'
📍 Top recommendations:
   • El Aguila
     Confidence: 1.00
   • The Flame
     Confidence: 0.90
------------------------------------------------------------


## Part 2: Spatial Intelligence & Relationships 📐

Unlike web search APIs, Camino provides deep spatial understanding. Let's explore distances, directions, and travel times between locations.

In [34]:
# Example 3: Spatial Relationships between NYC landmarks
landmarks = [
    {"name": "Empire State Building", "lat": 40.7484, "lon": -73.9857},
    {"name": "Central Park", "lat": 40.7829, "lon": -73.9654},
    {"name": "Brooklyn Bridge", "lat": 40.7061, "lon": -73.9969},
    {"name": "Statue of Liberty", "lat": 40.6892, "lon": -74.0445}
]

print("🗽 Spatial Relationships Between NYC Landmarks\n")
print("="*70)

# Calculate relationships between Empire State Building and other landmarks
start = landmarks[0]
print(f"Starting from: {start['name']}\n")

for destination in landmarks[1:]:
    try:
        relationship_request = RelationshipRequest(
            start=Coordinate(lat=start["lat"], lon=start["lon"]),
            end=Coordinate(lat=destination["lat"], lon=destination["lon"]),
            include=["distance", "direction", "travel_time", "description"]
        )
        
        relationship = client.relationship(relationship_request)
        
        print(f"📍 To {destination['name']}:")
        if hasattr(relationship, 'distance'):
            print(f"   📏 Distance: {relationship.distance}")
        if hasattr(relationship, 'actual_distance_km'):
            print(f"   📏 Actual distance: {relationship.actual_distance_km:.2f} km")
        if hasattr(relationship, 'direction'):
            print(f"   🧭 Direction: {relationship.direction}")
        if hasattr(relationship, 'walking_time'):
            print(f"   🚶 Walking time: {relationship.walking_time}")
        if hasattr(relationship, 'driving_time'):
            print(f"   🚗 Driving time: {relationship.driving_time}")
        if hasattr(relationship, 'description'):
            print(f"   📝 Description: {relationship.description}")
    except APIError as e:
        print(f"❌ Relationship calculation failed: {e.message}")
    print("-"*70)

🗽 Spatial Relationships Between NYC Landmarks

Starting from: Empire State Building

📍 To Central Park:
   📏 Distance: 4.2km
   📏 Actual distance: 6.89 km
   🧭 Direction: northeast
   🚶 Walking time: 59 minutes
   🚗 Driving time: 12 minutes
   📝 Description: 4.2km northeast of your location, about a 59 minutes walk
----------------------------------------------------------------------
📍 To Brooklyn Bridge:
   📏 Distance: 4.8km
   📏 Actual distance: 6.26 km
   🧭 Direction: south
   🚶 Walking time: 1h 13m
   🚗 Driving time: 14 minutes
   📝 Description: 4.8km south of your location, about a 1h 13m walk
----------------------------------------------------------------------
📍 To Statue of Liberty:
   📏 Distance: 8.2km
   📏 Actual distance: 13.27 km
   🧭 Direction: southwest
   🚶 Walking time: 1h 57m
   🚗 Driving time: 20 minutes
   📝 Description: 8.2km southwest of your location, about a 1h 57m walk
----------------------------------------------------------------------


## Part 3: Location Context Understanding 🏙️

Camino can analyze what's around a location, helping agents understand the environment and make informed decisions.

In [35]:
# Example 4: Location Context Understanding
locations_to_analyze = [
    {"name": "Times Square", "lat": 40.7589, "lon": -73.9851},
    {"name": "Wall Street", "lat": 40.7074, "lon": -74.0113},
    {"name": "Greenwich Village", "lat": 40.7336, "lon": -74.0027}
]

print("🏙️ Location Context Analysis\n")
print("="*70)

for loc in locations_to_analyze:
    try:
        # Get context about the location
        context_request = ContextRequest(
            location=Coordinate(lat=loc["lat"], lon=loc["lon"]),
            radius=500,
            categories=["restaurant", "entertainment", "shopping", "services"]
        )
        
        context_info = client.context(context_request)
        
        print(f"\n📍 {loc['name']}")
        print(f"Coordinates: ({loc['lat']:.4f}, {loc['lon']:.4f})\n")
        
        if hasattr(context_info, 'area_description'):
            print("🏘️ Area Description:")
            print(f"   {context_info.area_description}\n")
        
        if hasattr(context_info, 'search_radius'):
            print(f"⭕ Search radius: {context_info.search_radius}m")
        
        if hasattr(context_info, 'total_places_found'):
            print(f"📊 Total places found: {context_info.total_places_found}")
        
        if hasattr(context_info, 'relevant_places'):
            print("\n🏪 Nearby Amenities:")
            places = context_info.relevant_places
            if hasattr(places, 'restaurants') and places.restaurants:
                print(f"   • Restaurants: {', '.join(places.restaurants[:3])}")
            if hasattr(places, 'services') and places.services:
                print(f"   • Services: {', '.join(places.services[:3])}")
            if hasattr(places, 'shops') and places.shops:
                print(f"   • Shops: {', '.join(places.shops[:3])}")
            if hasattr(places, 'attractions') and places.attractions:
                print(f"   • Attractions: {', '.join(places.attractions[:3])}")
    
    except APIError as e:
        print(f"❌ Context request failed: {e.message}")
    
    print("-"*70)

🏙️ Location Context Analysis


📍 Times Square
Coordinates: (40.7589, -73.9851)

🏘️ Area Description:
   This area is a vibrant hub for entertainment and dining, boasting a high concentration of restaurants, theaters, and cafes. Its abundance of eateries and performance venues makes it a popular destination for leisure and cultural experiences.

⭕ Search radius: 500m
📊 Total places found: 852

🏪 Nearby Amenities:
   • Restaurants: Hard Rock Cafe, Junior's, Starbucks
   • Services: Chase, Chase, Hippodrome Parking
   • Shops: Key Food, AT&T, Unnamed
   • Attractions: Times Square Museum & Visitor Center, DoubleTree Suites, George M. Cohan
----------------------------------------------------------------------

📍 Wall Street
Coordinates: (40.7074, -74.0113)

🏘️ Area Description:
   This area appears to be a vibrant urban space, likely a plaza or thoroughfare, offering convenient seating, waste disposal, and a variety of dining options, including fast food, with ample bicycle parking.

⭕ Se

## Part 4: Journey Planning 🗺️

One of Camino's most powerful features is journey planning—perfect for agents that need to plan complex trips with multiple stops.

In [36]:
# Example 5: Multi-waypoint Journey Planning
print("🗺️ Planning a Perfect Tourist Day in NYC\n")
print("="*70)

# Define waypoints for our journey
waypoints = [
    Waypoint(lat=40.7589, lon=-73.9851, purpose="Start at Times Square for breakfast"),
    Waypoint(lat=40.7829, lon=-73.9654, purpose="Morning walk in Central Park"),
    Waypoint(lat=40.7614, lon=-73.9776, purpose="Visit Museum of Modern Art"),
    Waypoint(lat=40.7484, lon=-73.9857, purpose="Lunch and Empire State Building"),
    Waypoint(lat=40.7061, lon=-73.9969, purpose="Walk across Brooklyn Bridge")
]

try:
    # Plan the journey with constraints
    journey_request = JourneyRequest(
        waypoints=waypoints,
        constraints={
            "transport": "walking",
            "time_budget": "6 hours"
        }
    )
    
    journey_plan = client.journey(journey_request)
    
    print("📅 Journey Analysis:\n")
    
    if hasattr(journey_plan, 'feasible'):
        print(f"✅ Feasible: {journey_plan.feasible}")
    
    if hasattr(journey_plan, 'total_distance_km'):
        print(f"📏 Total distance: {journey_plan.total_distance_km:.2f} km")
    
    if hasattr(journey_plan, 'total_time_formatted'):
        print(f"⏱️ Total time: {journey_plan.total_time_formatted}")
    
    if hasattr(journey_plan, 'transport_mode'):
        print(f"🚶 Transport mode: {journey_plan.transport_mode}")
    
    if hasattr(journey_plan, 'route_segments') and journey_plan.route_segments:
        print(f"\n📍 Route segments: {len(journey_plan.route_segments)}")
        for i, segment in enumerate(journey_plan.route_segments, 1):
            if hasattr(segment, 'from_') and hasattr(segment, 'to'):
                print(f"\nSegment {i}: {segment.from_.purpose} → {segment.to.purpose}")
                if hasattr(segment, 'distance_km'):
                    print(f"   Distance: {segment.distance_km:.2f} km")
                if hasattr(segment, 'estimated_time'):
                    print(f"   Time: {segment.estimated_time}")
    
    if hasattr(journey_plan, 'analysis') and hasattr(journey_plan.analysis, 'summary'):
        print(f"\n💡 Analysis: {journey_plan.analysis.summary}")

except APIError as e:
    print(f"❌ Journey planning failed: {e.message}")

🗺️ Planning a Perfect Tourist Day in NYC

📅 Journey Analysis:

✅ Feasible: True
📏 Total distance: 12.14 km
⏱️ Total time: 1h 47m
🚶 Transport mode: walking

📍 Route segments: 4

Segment 1: Start at Times Square for breakfast → Morning walk in Central Park
   Distance: 3.14 km
   Time: 44 minutes

Segment 2: Morning walk in Central Park → Visit Museum of Modern Art
   Distance: 2.60 km
   Time: 39 minutes

Segment 3: Visit Museum of Modern Art → Lunch and Empire State Building
   Distance: 1.60 km
   Time: 24 minutes

Segment 4: Lunch and Empire State Building → Walk across Brooklyn Bridge
   Distance: 4.80 km
   Time: 1h 13m

💡 Analysis: This is a well-paced walking tour of iconic NYC landmarks, though the final leg is quite long.

**Optimization Suggestion:** Consider a subway ride for the final segment from the Empire State Building to the Brooklyn Bridge to save time and energy.

**Potential Issue/Consideration:** The total time of 107 minutes seems underestimated given the distances

## Part 5: Route Planning & Navigation 🚗

Camino provides detailed routing information using OSRM (Open Source Routing Machine) for different modes of transport.

In [37]:
# Example 6: Point-to-Point Route Planning
print("🚦 Route Planning: Central Park to Times Square\n")
print("="*70)

# Central Park to Times Square
start_coord = Coordinate(lat=40.7831, lon=-73.9712)
end_coord = Coordinate(lat=40.7589, lon=-73.9851)

transport_modes = ["foot", "bike", "car"]

for mode in transport_modes:
    try:
        route_request = RouteRequest(
            start_lat=start_coord.lat,
            start_lon=start_coord.lon,
            end_lat=end_coord.lat,
            end_lon=end_coord.lon,
            mode=mode,
            include_geometry=False # Set this to True if you're showing a map to a user
        )
        
        route = client.route(route_request)
        
        print(f"\n🚗 Mode: {mode.upper()}")
        
        if hasattr(route, 'summary'):
            summary = route.summary
            if hasattr(summary, 'total_distance_meters'):
                print(f"   📏 Distance: {summary.total_distance_meters / 1000:.2f} km")
            if hasattr(summary, 'total_duration_seconds'):
                minutes = summary.total_duration_seconds / 60
                print(f"   ⏱️ Duration: {minutes:.1f} minutes")
        
        if hasattr(route, 'instructions') and route.instructions:
            print(f"   🗺️ Instructions: {len(route.instructions)} steps")
            print("   First steps:")
            for i, instruction in enumerate(route.instructions[:3], 1):
                print(f"      {i}. {instruction}")
        
        if hasattr(route, 'include_geometry'):
            print(f"   📍 Geometry included: {route.include_geometry}")
        
    except APIError as e:
        print(f"\n❌ Route calculation for {mode} failed: {e.message}")
    
    print("-"*70)

🚦 Route Planning: Central Park to Times Square


🚗 Mode: FOOT
   📏 Distance: 3.00 km
   ⏱️ Duration: 40.0 minutes
   🗺️ Instructions: 6 steps
   First steps:
      1. Depart right and travel for 436.7 meters.
      2. Turn left and travel for 25.3 meters.
      3. End of road right onto Central Park Outer Loop and travel for 1392.7 meters.
   📍 Geometry included: True
----------------------------------------------------------------------

🚗 Mode: BIKE
   📏 Distance: 3.20 km
   ⏱️ Duration: 14.5 minutes
   🗺️ Instructions: 11 steps
   First steps:
      1. Depart right and travel for 38.9 meters.
      2. Turn left onto West 82nd Street and travel for 11.1 meters.
      3. Turn right onto Central Park West and travel for 823.7 meters.
   📍 Geometry included: True
----------------------------------------------------------------------

🚗 Mode: CAR
   📏 Distance: 3.74 km
   ⏱️ Duration: 7.2 minutes
   🗺️ Instructions: 7 steps
   First steps:
      1. Depart right onto Central Park West and

## Part 6: MCP Integration for AI Agents 🔌

Camino supports the Model Context Protocol (MCP), allowing seamless integration with AI development environments like Claude Desktop and Cursor IDE.

In [ ]:
# MCP Configuration Examples
print("\n🔧 MCP Configuration Examples\n")
print("="*70)

# Claude Desktop configuration
claude_config = {
    "mcpServers": {
        "camino-ai": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-http", 
                    f"https://mcp.getcamino.ai/mcp?caminoApiKey={api_key}"]
        }
    }
}

print("📱 Claude Desktop Configuration:")
print("Add this to your claude_desktop_config.json:")
print(json.dumps(claude_config, indent=2))

print("\n" + "-"*70)

# Cursor IDE configuration
cursor_config = {
    "mcpServers": {
        "camino-ai": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-http",
                    f"https://mcp.getcamino.ai/mcp?caminoApiKey={api_key}"]
        }
    }
}

print("\n💻 Cursor IDE Configuration:")
print("Add this to your .cursor/config.json:")
print(json.dumps(cursor_config, indent=2))

## Conclusion & Next Steps 🎯

Congratulations! You've learned how to use Camino AI to build location-aware AI agents with capabilities including:

### What You've Learned:
- ✅ **Natural language location queries** with AI-powered understanding
- ✅ **Spatial intelligence** for distances, directions, and relationships
- ✅ **Context-aware search** that understands intent and preferences
- ✅ **Route planning** with multi-modal transportation options
- ✅ **Journey optimization** for complex multi-stop trips
- ✅ **MCP integration** for seamless AI agent development

### Key Advantages of Camino AI:
1. **🌍 Physical World Grounding**: Connect AI to real locations and places
2. **💰 Cost-Effective**: only $0.001 per call with a generous free tier
3. **🧠 AI-Native**: Built specifically for AI agents, not traditional apps
4. **🔌 Easy Integration**: Native MCP support for popular AI tools
5. **📊 Comprehensive**: All location intelligence in one API

### Next Steps:
1. **Get Your API Key**: Sign up at [app.getcamino.ai](https://app.getcamino.ai?utm_source=agents-towards-production&utm_medium=github&utm_campaign=tutorial)
2. **Explore the API**: Try the interactive playground
3. **Build Your Agent**: Combine Camino with other tools (like Tavily for web search)
4. **Join the Community**: Share your location-aware agents on Github or LinkedIn

### Combining with Other Tools:
Camino AI works great alongside other agent tools:
- **Tavily**: For web search and content extraction
- **LangChain/LlamaIndex**: For orchestration and memory
- **OpenAI/Anthropic**: For reasoning and decision-making
- **Vector Databases**: For location-based knowledge storage

### Resources:
- 📚 [API Documentation](https://api.getcamino.ai/docs)
- 🎮 [Interactive Playground](https://app.getcamino.ai/playground?utm_source=agents-towards-production&utm_medium=github&utm_campaign=tutorial)
---

**Ready to build location-aware AI agents?** Start with Camino AI and give your agents the power to understand and navigate the physical world! 🚀